# IMPORT PACKAGES

In [4]:
import glob
import pandas as pd
import math
import glob
import pandas as pd
import numpy as np
from pandas import DataFrame
import numpy as np
import pandas as pd
import os
import numpy as np
import pandas as pd
import os
import glob
from sklearn.covariance import EmpiricalCovariance
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import LedoitWolf
from sklearn.preprocessing import MinMaxScaler
from npeet import entropy_estimators as ee
from pyinform import transfer_entropy 
import neurokit2 as nk
from scipy import stats
import glob
from sklearn.covariance import EmpiricalCovariance
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import LedoitWolf
from sklearn.preprocessing import MinMaxScaler
from npeet import entropy_estimators as ee
from pyinform import transfer_entropy 
import neurokit2 as nk
from scipy import stats
from numpy import matrix 
from sklearn.model_selection import StratifiedKFold
from sklearn import model_selection
from sklearn import svm
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
import antropy as ant


# LOAD DATASET

In [5]:
dystonia= [pd.read_csv(n,sep="\t", index_col=False,header=None, skiprows=[0]) for n in glob.glob(r'PATH FOLDER/*')]
healthy= [pd.read_csv(n,sep="\t", index_col=False,header=None, skiprows=[0]) for n in glob.glob(r'PATH FOLDER/*')]


# DEFINE MINIMUM SIZE

In [6]:
list_dystonia=[]
list_healthy=[]

for i in dystonia:
    list_dystonia.append(len(i))

for i in healthy:
    list_healthy.append(len(i))

print(min(list_dystonia))
print(min(list_healthy))

45192
51917


# DEFINE SOME FUNCTIONS

In [11]:
min_max_scaler = MinMaxScaler()
estimator1 = EmpiricalCovariance()
estimator2 = LedoitWolf()
def flatten_matrix(x):
    test1 = np.array(x)
    flatten= matrix.flatten(test1)
    return flatten

def replace(a):
    y= np.matrix(a)
       
    y1 = a.replace('nan', 0, regex=True)
    y2 = y1.values

    y3 = y2.astype(float)
    y4= np.matrix(y3)
    return y4

def flatten_matrix(x):
    test1 = np.array(x)
    #y = test1.astype(np.float)
    #y= np.fromstring( test1, dtype=np.float )
    flatten= matrix.flatten(test1)
    return flatten
def threshold1(matrix):
    list_min = []
    list_max = []
    matrix1 = np.array(matrix, dtype=np.float)
    threshold, upper, lower = 0.5, 1, 0
    matrix1[matrix1 <= threshold] = lower
    matrix1[matrix1 > threshold] = upper
    return matrix1


# EXPORT DYSTONIA MATRICES AUTOMATICALLY

In [ ]:
# Modify the window sizes and corresponding folder names
window_sizes = [1506]  # Add more sizes as needed
window_names = ['10s']  # Corresponding window folder names

channels = 32 # add number of channel

# Create output directory
output_dir = 'matrices-final/dystonia/'
os.makedirs(output_dir, exist_ok=True)
# Loop through different window sizes and names

for window, window_name in zip(window_sizes, window_names):
    window_dir = os.path.join(output_dir, f'window_{window_name}')
    os.makedirs(window_dir, exist_ok=True)

            # Loop through each patient's EEG data
    z=0
    while z< len(dystonia):
        for t in dystonia:
            t= t[0:min(list_dystonia)]
            N = len(t)
            # Calculate the number of intervals without overlap
            num_intervals = N // window
        
            # Loop through each interval for this patient
            for k in range(num_intervals):
        
                start = k * window
                end = start + window

                # Extract the interval data
                interval_data = t.iloc[start:end, :].values
                 
                 
                # Calculate Spearman correlation matrix for the interval data
                spearman_matrix, _ = stats.spearmanr(interval_data)

                # Calculate Pearson correlation matrix for the interval data
                pearson_matrix = np.corrcoef(interval_data, rowvar=False)

                # Calculate Canonical correlation matrix for the interval data
                estimator1.fit(interval_data)
                canonical_matrix = estimator1.covariance_

                # Calculate LW correlation matrix for the interval data
                estimator2.fit(interval_data)
                lw_matrix = estimator2.covariance_

                # Calculate TE correlation matrix for the interval data
                interval_data_normalized = min_max_scaler.fit_transform(interval_data)
                interval_data_threshold = threshold1(interval_data_normalized)

                M = np.zeros((channels, channels))
                for i in range(channels):
                    P1 = interval_data_threshold[i]
                    for j in range(i + 1, channels):
                        P2 = interval_data_threshold[j]

                        try:
                            M[i][j] = M[j][i] = transfer_entropy(P1, P2, k=2)
                        except ValueError:  # raised if `y` is empty.
                            pass
                    M[i][i] = 0.

                # Calculate MI correlation matrix for the interval data
                M1 = np.zeros((channels, channels))
                for i in range(channels):
                    P1 = interval_data_threshold[i]
                    for j in range(i + 1, channels):
                        P2 = interval_data_threshold[j]

                        try:
                            M1[i][j] = M1[j][i] = nk.mutual_information(P1, P2, method="varoquaux")
                        except ValueError:  # raised if `y` is empty.
                            pass
                    M1[i][i] = 0.5
            
                    # Save TE correlation matrix to CSV
                te_dir = os.path.join(window_dir, 'TE')  # Directory for TE matrices
                os.makedirs(te_dir, exist_ok=True)
                te_df = pd.DataFrame(M)
                te_df.to_csv(os.path.join(te_dir, f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)

                    # Save MI correlation matrix to CSV
                mi_dir = os.path.join(window_dir, 'MI')  # Directory for MI matrices
                os.makedirs(mi_dir, exist_ok=True)
                mi_df = pd.DataFrame(M1)
                mi_df.to_csv(os.path.join(mi_dir, f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)

                 #Save LW correlation matrix to CSV
                lw_dir = os.path.join(window_dir, 'LW')
                os.makedirs(lw_dir, exist_ok=True)
                lw_df = pd.DataFrame(lw_matrix)
                lw_df.to_csv(os.path.join(lw_dir, f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)

                # Save Canonical correlation matrix to CSV
                canonical_dir = os.path.join(window_dir, 'canonical')
                os.makedirs(canonical_dir, exist_ok=True)
                canonical_df = pd.DataFrame(canonical_matrix)
                canonical_df.to_csv(os.path.join(canonical_dir, f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)

                # Save Spearman correlation matrix to CSV
                spearman_dir = os.path.join(window_dir, 'spearman')
                os.makedirs(spearman_dir, exist_ok=True)
                spearman_df = pd.DataFrame(spearman_matrix)
                spearman_df.to_csv(os.path.join(spearman_dir, f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)

                # Save Pearson correlation matrix to CSV
                pearson_dir = os.path.join(window_dir, 'pearson')
                os.makedirs(pearson_dir, exist_ok=True)
                pearson_df = pd.DataFrame(pearson_matrix)
                pearson_df.to_csv(os.path.join(pearson_dir,f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)    
                
                z= z+1    

# EXPORT HEALTHY MATRICES AUTOMATICALLY

In [ ]:
# Create output directory
output_dir = 'matrices-final/healthy/'
os.makedirs(output_dir, exist_ok=True)
# Loop through different window sizes and names

for window, window_name in zip(window_sizes, window_names):
    window_dir = os.path.join(output_dir, f'window_{window_name}')
    os.makedirs(window_dir, exist_ok=True)

            # Loop through each patient's EEG data
    z=0
    while z< len(healthy):
        for t in healthy:
            t= t[0:min(list_healthy)]
            N = len(t)
            # Calculate the number of intervals without overlap
            num_intervals = N // window
        
            # Loop through each interval for this patient
            for k in range(num_intervals):
        
                start = k * window
                end = start + window

                # Extract the interval data
                interval_data = t.iloc[start:end, :].values
                 
                 
                # Calculate Spearman correlation matrix for the interval data
                spearman_matrix, _ = stats.spearmanr(interval_data)

                # Calculate Pearson correlation matrix for the interval data
                pearson_matrix = np.corrcoef(interval_data, rowvar=False)

                # Calculate Canonical correlation matrix for the interval data
                estimator1.fit(interval_data)
                canonical_matrix = estimator1.covariance_

                # Calculate LW correlation matrix for the interval data
                estimator2.fit(interval_data)
                lw_matrix = estimator2.covariance_

                # Calculate TE correlation matrix for the interval data
                interval_data_normalized = min_max_scaler.fit_transform(interval_data)
                interval_data_threshold = threshold1(interval_data_normalized)

                M = np.zeros((channels, channels))
                for i in range(channels):
                    P1 = interval_data_threshold[i]
                    for j in range(i + 1, channels):
                        P2 = interval_data_threshold[j]

                        try:
                            M[i][j] = M[j][i] = transfer_entropy(P1, P2, k=2)
                        except ValueError:  # raised if `y` is empty.
                            pass
                    M[i][i] = 0.

                # Calculate MI correlation matrix for the interval data
                M1 = np.zeros((channels, channels))
                for i in range(channels):
                    P1 = interval_data_threshold[i]
                    for j in range(i + 1, channels):
                        P2 = interval_data_threshold[j]

                        try:
                            M1[i][j] = M1[j][i] = nk.mutual_information(P1, P2, method="varoquaux")
                        except ValueError:  # raised if `y` is empty.
                            pass
                    M1[i][i] = 0.5
            
                    # Save TE correlation matrix to CSV
                te_dir = os.path.join(window_dir, 'TE')  # Directory for TE matrices
                os.makedirs(te_dir, exist_ok=True)
                te_df = pd.DataFrame(M)
                te_df.to_csv(os.path.join(te_dir, f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)

                    # Save MI correlation matrix to CSV
                mi_dir = os.path.join(window_dir, 'MI')  # Directory for MI matrices
                os.makedirs(mi_dir, exist_ok=True)
                mi_df = pd.DataFrame(M1)
                mi_df.to_csv(os.path.join(mi_dir, f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)

                 #Save LW correlation matrix to CSV
                lw_dir = os.path.join(window_dir, 'LW')
                os.makedirs(lw_dir, exist_ok=True)
                lw_df = pd.DataFrame(lw_matrix)
                lw_df.to_csv(os.path.join(lw_dir, f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)

                # Save Canonical correlation matrix to CSV
                canonical_dir = os.path.join(window_dir, 'canonical')
                os.makedirs(canonical_dir, exist_ok=True)
                canonical_df = pd.DataFrame(canonical_matrix)
                canonical_df.to_csv(os.path.join(canonical_dir, f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)

                # Save Spearman correlation matrix to CSV
                spearman_dir = os.path.join(window_dir, 'spearman')
                os.makedirs(spearman_dir, exist_ok=True)
                spearman_df = pd.DataFrame(spearman_matrix)
                spearman_df.to_csv(os.path.join(spearman_dir, f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)

                # Save Pearson correlation matrix to CSV
                pearson_dir = os.path.join(window_dir, 'pearson')
                os.makedirs(pearson_dir, exist_ok=True)
                pearson_df = pd.DataFrame(pearson_matrix)
                pearson_df.to_csv(os.path.join(pearson_dir,f'patient_{str(z)}_interval_{str(k)}_te.csv'), sep=',', header=False, index=False)    
                
                z= z+1    